# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [32]:
from azureml.core import Dataset, Environment, Experiment, Workspace

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.train.hyperdrive import normal, uniform, choice
import joblib
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

from azureml.widgets import RunDetails

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [33]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive-capstone'

experiment=Experiment(ws, experiment_name)

In [34]:
# fetch dataset
dataset = Dataset.get_by_name(ws, name='uci_heart_disease')
uci_heart_disease = dataset.to_pandas_dataframe()

# verify dataset
uci_heart_disease.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
# exploratory data analysis


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [35]:
# Create compute cluster to run HyperDrive
cluster_name = 'govindk-nvidia-gpunode'
try:
    compute_target = ComputeTarget(ws, cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC24', min_nodes=1, max_nodes=50)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Running


In [36]:
from azureml.core.environment import CondaDependencies, Environment

from azureml.core import Workspace, Experiment, ScriptRunConfig


myenv = Environment(name="govindk")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("scikit-learn")
myenv = Environment.from_pip_requirements('govindk', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/validated_linux_requirements.txt')

# set up script run configuration
config = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=compute_target,
    environment=myenv,
    arguments=['--input-data', dataset.id],
)



In [37]:
#TODO: Create the different params that you will be using during training
# The two parameters that we are tuning are
# n-estimators: The number of trees in the forest, a larger forest can lead to better performance
# max-depth: The maximum depth of the tree. It is important to find the right depth to balance generalization/overfitting
param_sampling = RandomParameterSampling(
   parameter_space={
       "--C": uniform(0.05, 0.1),
       "--max_iter": choice(16, 32, 64, 128)
    }
)

#TODO: Create your estimator and hyperdrive config
estimator = est = SKLearn(
    source_directory="./",
    entry_script="train.py",
    script_params={'--input-data': dataset.id, '--n_jobs': 30},
    compute_target=compute_target
)

early_termination = BanditPolicy(slack_factor = 0.1, evaluation_interval=2, delay_evaluation=5) 

hyperdrive_run_config = HyperDriveConfig(
    primary_metric_name="accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=20,
    hyperparameter_sampling=param_sampling,
    policy=early_termination,
    run_config=config
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [38]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [39]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=False)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

{'runId': 'HD_0a690927-2543-41b1-a799-f0d2ea16c11b',
 'target': 'govindk-nvidia-gpunode',
 'status': 'Completed',
 'startTimeUtc': '2021-08-23T21:08:39.53081Z',
 'endTimeUtc': '2021-08-23T21:12:12.905542Z',
 'properties': {'primary_metric_config': '{"name": "accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '4a8ccd2f-7f0e-4472-8a7a-51c7916b5e12',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.32.0',
  'score': '0.9340659340659341',
  'best_child_run_id': 'HD_0a690927-2543-41b1-a799-f0d2ea16c11b_7',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://kgovindmstoragec045ce3ac.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_0a690927-2543-41b1-a799-f0d2ea16c11b/azureml-logs/hyperdrive.tx

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [40]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run.get_metrics()

{'--C (Regularization Strength):': 0.09196444981833907,
 '--max_iter (Max iterations:': 128,
 '--n_jobs (CPU cores:': 1,
 'accuracy': 0.9340659340659341,
 'Confusion Matrix': 'aml://artifactId/ExperimentRun/dcid.HD_0a690927-2543-41b1-a799-f0d2ea16c11b_7/Confusion Matrix'}

In [41]:
#TODO: Save the best model
model = best_run.register_model(model_name='model', model_path='outputs/model.pkl')
print(f"Model {model.name}.v{model.version} correctly saved")

Model model.v2 correctly saved


In [ ]:
# Deprovision and delete the AmlCompute target. 
compute_target.delete()
print('Cluster Deleted')